In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# Any results you write to the current directory are saved as output.

/kaggle/input/hdfc-hackerearth-dataset/Test.csv
/kaggle/input/hdfc-hackerearth-dataset/Train.csv


In [2]:

import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score  
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier


In [3]:
train = pd.read_csv('/kaggle/input/hdfc-hackerearth-dataset/Train.csv')
test = pd.read_csv('/kaggle/input/hdfc-hackerearth-dataset/Test.csv')

In [4]:
train.shape

(17521, 2395)

In [5]:
#cm

## Duplicate the data for class 1

In [6]:
concat_df =  train[train['Col2']==1].sample(700)

In [7]:
train = pd.concat(objs=[train, concat_df], axis=0)

In [8]:
class_0 =len(train[train['Col2']==0])
class_1 =len(train[train['Col2']==1])

print(class_0)
print(class_1)
print(class_0/(class_0+class_1))
print(class_1/(class_0+class_1))

15760
2461
0.8649360627846989
0.13506393721530102


In [9]:
target = train['Col2']
features  = train.columns.tolist()
features.remove('Col1')
features.remove('Col2')
train_input  = train[features] 

In [10]:
def remove_null_column(df,percentage = 60):
    rows = df.shape[0]
    null_percantage_columns = (df.isna().sum()/rows)*100
    null_percantage_columns = pd.DataFrame(null_percantage_columns).reset_index()
    null_percantage_columns.columns = ['columns','percentage_null']
    columns_removed = null_percantage_columns[null_percantage_columns['percentage_null']>=percentage]
    null_percantage_columns = null_percantage_columns[null_percantage_columns['percentage_null']<percentage]
    
    df = df[null_percantage_columns['columns'].tolist()]
    return df ,columns_removed




In [11]:
train_null_removed,columns_removed = remove_null_column(train_input,50)
test_null_removed = test[train_null_removed.columns.tolist()]

In [12]:
def thershold_matrix(model,x_text,Y_test):	
	pred_proba_df = pd.DataFrame(model.predict_proba(x_test))
	threshold_list = [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,.7,.75,.8,.85,.9,.95,.99]
	columns = ['Threshold', 'Accuracy', 'Precision','Recall','F1','TN','FP','FN','TP']
	score_df=pd.DataFrame(columns=columns) 
	for i in threshold_list:
		Y_test_pred = pred_proba_df.applymap(lambda x: 1 if x>i else 0)
		test_accuracy = metrics.accuracy_score(Y_test.as_matrix().reshape(Y_test.as_matrix().size,1),
											Y_test_pred.iloc[:,1].as_matrix().reshape(Y_test_pred.iloc[:,1].as_matrix().size,1))
		test_precision = metrics.precision_score(Y_test.as_matrix().reshape(Y_test.as_matrix().size,1), Y_test_pred.iloc[:,1].as_matrix().reshape(Y_test_pred.iloc[:,1].as_matrix().size,1))
		test_recall = metrics.recall_score(Y_test.as_matrix().reshape(Y_test.as_matrix().size,1), Y_test_pred.iloc[:,1].as_matrix().reshape(Y_test_pred.iloc[:,1].as_matrix().size,1))
		test_f1 = metrics.f1_score(Y_test.as_matrix().reshape(Y_test.as_matrix().size,1), Y_test_pred.iloc[:,1].as_matrix().reshape(Y_test_pred.iloc[:,1].as_matrix().size,1),average = 'weighted')
		TN, FP, FN, TP = confusion_matrix(Y_test.as_matrix().reshape(Y_test.as_matrix().size,1), Y_test_pred.iloc[:,1].as_matrix().reshape(Y_test_pred.iloc[:,1].as_matrix().size,1)).ravel()
		score_df.loc[len(score_df)]=[i,test_accuracy,test_precision,test_recall,test_f1,TN, FP, FN, TP]
	return score_df
		

In [13]:
#dataset = [train_null_removed,test_null_removed]

In [14]:
cnt = 0
cat_column = []
cont_column = []
uniques = dict()
for column in train_null_removed.columns:
    unique = len(train_null_removed[column].unique())/int(train_null_removed.shape[0])
    unique = unique*100
    uniques[column] = unique
    if unique < 0.05:
        train_null_removed[column].fillna(train_null_removed[column].mode()[0], inplace = True)
        test_null_removed[column].fillna(test_null_removed[column].mode()[0], inplace = True)
        cnt = cnt+1
        cat_column.append(column)
    else:
        train_null_removed[column].fillna(train_null_removed[column].mean(), inplace = True)
        test_null_removed[column].fillna(test_null_removed[column].mean(), inplace = True)
        cont_column.append(column)

In [15]:
print(len(cat_column))
print(len(cont_column))

1543
628


In [16]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(train_null_removed,target,test_size = .25, random_state = 24,stratify=target)

In [17]:
lg_fs = LogisticRegression()

In [18]:
lg_fs.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [19]:
lg_fs.score(x_test,y_test)

0.8612818261633012

In [20]:
logistic_score_df = thershold_matrix(lg_fs,x_test,y_test)

In [21]:
logistic_score_df[logistic_score_df['F1']==logistic_score_df.F1.max()]

,Threshold,Accuracy,Precision,Recall,F1,TN,FP,FN,TP
9,0.5,0.861282,0.345455,0.030894,0.807911,3905.0,36.0,596.0,19.0


In [22]:
y_pred = lg_fs.predict(test_null_removed)
X = test.Col1
df = pd.DataFrame({'Col1': X,'Col2': y_pred})
df.to_csv('Linear_regress_Submission.csv',index= False)

Random Forest

In [23]:
model = RandomForestClassifier(n_estimators=1000,verbose = 2,n_jobs=10)
model.fit(x_train,y_train)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.


building tree 1 of 1000
building tree 2 of 1000
building tree 3 of 1000
building tree 4 of 1000
building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000
building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000
building tree 13 of 1000
building tree 14 of 1000
building tree 15 of 1000
building tree 16 of 1000
building tree 17 of 1000
building tree 18 of 1000
building tree 19 of 1000
building tree 20 of 1000
building tree 21 of 1000
building tree 22 of 1000
building tree 23 of 1000
building tree 24 of 1000
building tree 25 of 1000
building tree 26 of 1000
building tree 27 of 1000
building tree 28 of 1000
building tree 29 of 1000
building tree 30 of 1000
building tree 31 of 1000


[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:    3.0s


building tree 32 of 1000
building tree 33 of 1000
building tree 34 of 1000
building tree 35 of 1000
building tree 36 of 1000
building tree 37 of 1000
building tree 38 of 1000
building tree 39 of 1000
building tree 40 of 1000
building tree 41 of 1000
building tree 42 of 1000
building tree 43 of 1000
building tree 44 of 1000
building tree 45 of 1000
building tree 46 of 1000
building tree 47 of 1000
building tree 48 of 1000
building tree 49 of 1000
building tree 50 of 1000
building tree 51 of 1000
building tree 52 of 1000
building tree 53 of 1000
building tree 54 of 1000
building tree 55 of 1000
building tree 56 of 1000
building tree 57 of 1000
building tree 58 of 1000
building tree 59 of 1000
building tree 60 of 1000
building tree 61 of 1000
building tree 62 of 1000
building tree 63 of 1000
building tree 64 of 1000
building tree 65 of 1000
building tree 66 of 1000
building tree 67 of 1000
building tree 68 of 1000
building tree 69 of 1000
building tree 70 of 1000
building tree 71 of 1000


[Parallel(n_jobs=10)]: Done 142 tasks      | elapsed:   17.4s


...

building tree 357 of 1000
building tree 358 of 1000
building tree 359 of 1000
building tree 360 of 1000
building tree 361 of 1000
building tree 362 of 1000
building tree 363 of 1000
building tree 364 of 1000
building tree 365 of 1000
building tree 366 of 1000
building tree 367 of 1000
building tree 368 of 1000
building tree 369 of 1000
building tree 370 of 1000
building tree 371 of 1000
building tree 372 of 1000
building tree 373 of 1000
building tree 374 of 1000
building tree 375 of 1000
building tree 376 of 1000
building tree 377 of 1000
building tree 378 of 1000
building tree 379 of 1000
building tree 380 of 1000building tree 381 of 1000

building tree 382 of 1000
building tree 383 of 1000
building tree 384 of 1000
building tree 385 of 1000
building tree 386 of 1000
building tree 387 of 1000
building tree 388 of 1000
building tree 389 of 1000
building tree 390 of 1000
building tree 391 of 1000
building tree 392 of 1000
building tree 393 of 1000
building tree 394 of 1000building tree

[Parallel(n_jobs=10)]: Done 628 tasks      | elapsed:  1.3min


building tree 638 of 1000
building tree 639 of 1000
building tree 640 of 1000
building tree 641 of 1000
building tree 642 of 1000
building tree 643 of 1000
building tree 644 of 1000
building tree 645 of 1000
building tree 646 of 1000
building tree 647 of 1000
building tree 648 of 1000
building tree 649 of 1000
building tree 650 of 1000
building tree 651 of 1000
building tree 652 of 1000
building tree 653 of 1000
building tree 654 of 1000
building tree 655 of 1000
building tree 656 of 1000
building tree 657 of 1000
building tree 658 of 1000
building tree 659 of 1000
building tree 660 of 1000
building tree 661 of 1000
building tree 662 of 1000
building tree 663 of 1000
building tree 664 of 1000
building tree 665 of 1000
building tree 666 of 1000
building tree 667 of 1000
building tree 668 of 1000
building tree 669 of 1000
building tree 670 of 1000
building tree 671 of 1000
building tree 672 of 1000
building tree 673 of 1000
building tree 674 of 1000
building tree 675 of 1000
building tre

[Parallel(n_jobs=10)]: Done 1000 out of 1000 | elapsed:  2.0min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=10, oob_score=False, random_state=None, verbose=2,
                       warm_start=False)

In [24]:
model.score(x_test,y_test) #prev 0.91544

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:    0.0s
[Parallel(n_jobs=10)]: Done 142 tasks      | elapsed:    0.1s
[Parallel(n_jobs=10)]: Done 345 tasks      | elapsed:    0.3s
[Parallel(n_jobs=10)]: Done 628 tasks      | elapsed:    0.6s
[Parallel(n_jobs=10)]: Done 1000 out of 1000 | elapsed:    0.9s finished


0.9216417910447762

In [25]:
#features_importance = list(model.feature_importances_)
#print(np.max(features_importance))

In [26]:
print(np.max(features_importance))

NameError: name 'features_importance' is not defined

In [27]:
rm_df = thershold_matrix(model,x_test,y_test)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:    0.0s
[Parallel(n_jobs=10)]: Done 142 tasks      | elapsed:    0.1s
[Parallel(n_jobs=10)]: Done 345 tasks      | elapsed:    0.3s
[Parallel(n_jobs=10)]: Done 628 tasks      | elapsed:    0.6s
[Parallel(n_jobs=10)]: Done 1000 out of 1000 | elapsed:    0.9s finished


In [28]:
rm_df[rm_df['F1']==rm_df.F1.max()]

,Threshold,Accuracy,Precision,Recall,F1,TN,FP,FN,TP
8,0.45,0.920983,0.932203,0.447154,0.908632,3921.0,20.0,340.0,275.0


In [29]:
pred_proba_df = pd.DataFrame(model.predict_proba(test_null_removed))
Y_test_pred = pred_proba_df.applymap(lambda x: 1 if x>0.45 else 0)
y_pred = Y_test_pred.iloc[:,1]
X = test.Col1
df = pd.DataFrame({'Col1': X,'Col2': y_pred})
df.to_csv('RandomForest_90_.csv',index= False)
from IPython.display import FileLink
FileLink(r'RandomForest_90_.csv')

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:    0.1s
[Parallel(n_jobs=10)]: Done 142 tasks      | elapsed:    0.5s
[Parallel(n_jobs=10)]: Done 345 tasks      | elapsed:    1.1s
[Parallel(n_jobs=10)]: Done 628 tasks      | elapsed:    2.0s
[Parallel(n_jobs=10)]: Done 1000 out of 1000 | elapsed:    3.1s finished


/kaggle/working/RandomForest_90_.csv

In [30]:
param_grid = {
    'bootstrap': [True],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

NameError: name 'RandomForestRegressor' is not defined

## Confusion matrix

In [31]:
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
print('Accuracy Score : ' + str(accuracy_score(y_test,lg_fs.predict(x_test))))
print('Precision Score : ' + str(precision_score(y_test,lg_fs.predict(x_test))))
print('Recall Score : ' + str(recall_score(y_test,lg_fs.predict(x_test))))
print('F1 Score : ' + str(f1_score(y_test,lg_fs.predict(x_test))))

Accuracy Score : 0.8612818261633012
Precision Score : 0.34545454545454546
Recall Score : 0.030894308943089432
F1 Score : 0.056716417910447764


In [32]:

print('Confusion Matrix : \n' + str(confusion_matrix(y_test,lg_fs.predict(x_test))))

Confusion Matrix : 
[[3905   36]
 [ 596   19]]


In [33]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
x = pca.fit_transform(train_null_removed)

In [34]:
explained_variance=pca.explained_variance_ratio_
explained_variance

array([0.96093411, 0.02969953, 0.00500398])

In [35]:
explained_variance.sum()

0.995637615623037

In [36]:
x_train,x_test,y_train,y_test = train_test_split(x,target,test_size = .30, random_state = 24,stratify=target)

In [37]:
lg_PCA = LogisticRegression()

In [38]:
lg_PCA.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [39]:
lg_PCA.score(x_test,y_test)

0.4020486555697823

In [40]:
logistic_PCA_score_df = thershold_matrix(lg_PCA,x_test,y_test)

In [41]:
logistic_PCA_score_df[logistic_PCA_score_df['F1']==logistic_PCA_score_df.F1.max()]

,Threshold,Accuracy,Precision,Recall,F1,TN,FP,FN,TP
12,0.65,0.864094,0.333333,0.006775,0.803673,4719.0,10.0,733.0,5.0


In [42]:
logistic_PCA_score_df.sort_values('F1',ascending =False)

,Threshold,Accuracy,Precision,Recall,F1,TN,FP,FN,TP
12,0.65,0.864094,0.333333,0.006775,0.803673,4719.0,10.0,733.0,5.0
17,0.90,0.865008,0.500000,0.004065,0.803450,4726.0,3.0,735.0,3.0
11,0.60,0.863545,0.277778,0.006775,0.803393,4716.0,13.0,733.0,5.0
13,0.70,0.864094,0.307692,0.005420,0.803331,4720.0,9.0,734.0,4.0
16,0.85,0.864459,0.333333,0.004065,0.803172,4723.0,6.0,735.0,3.0
10,0.55,0.862447,0.230769,0.008130,0.803167,4709.0,20.0,732.0,6.0
19,0.99,0.865008,0.500000,0.002710,0.803101,4727.0,2.0,736.0,2.0
14,0.75,0.864277,0.300000,0.004065,0.803079,4722.0,7.0,735.0,3.0
15,0.80,0.864277,0.300000,0.004065,0.803079,4722.0,7.0,735.0,3.0
18,0.95,0.864825,0.400000,0.002710,0.803009,4726.0,3.0,736.0,2.0


In [43]:
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,lg_PCA.predict(x_test))))

Confusion Matrix : 
[[1629 3100]
 [ 169  569]]


In [44]:
x_test = pca.fit_transform(test_null_removed)

In [45]:
x_test.shape

(20442, 3)

In [46]:
y_pred = lg_PCA.predict(x_test)
X = test.Col1
df = pd.DataFrame({'Col1': X,'Col2': y_pred})
df.to_csv('Linear_regress_PCA_Submission.csv',index= False)

In [47]:

train_objs_num = len(train_null_removed)

dataset = pd.concat(objs=[train_null_removed, test_null_removed], axis=0)
dataset_preprocessed = pd.get_dummies(dataset,columns = cat_column)
train_preprocessed = dataset_preprocessed[:train_objs_num]
test_preprocessed = dataset_preprocessed[train_objs_num:]
print(train_preprocessed.shape)
print(test_preprocessed.shape)

(18221, 9605)
(20442, 9605)


In [48]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_preprocessed[cont_column] = scaler.fit_transform(train_preprocessed[cont_column])
test_preprocessed[cont_column] = scaler.fit_transform(test_preprocessed[cont_column])




In [49]:
x_train,x_test,y_train,y_test = train_test_split(train_preprocessed,target,test_size = .25, random_state = 24,stratify=target)

In [50]:
lg_preprocessed = LogisticRegression()
lg_preprocessed.fit(x_train,y_train)



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [51]:
lg_preprocessed.score(x_test,y_test)

0.8485513608428445

In [52]:
lg_preprocessed_df = thershold_matrix(lg_preprocessed,x_test,y_test)


In [53]:
lg_preprocessed_df[lg_preprocessed_df['F1']==lg_preprocessed_df.F1.max()]

,Threshold,Accuracy,Precision,Recall,F1,TN,FP,FN,TP
9,0.5,0.848551,0.424547,0.343089,0.841633,3655.0,286.0,404.0,211.0


In [54]:
#y_pred = lg_preprocessed.predict(test_preprocessed)
pred_proba_df = pd.DataFrame(lg_preprocessed.predict_proba(test_preprocessed))
Y_test_pred = pred_proba_df.applymap(lambda x: 1 if x>0.5 else 0)
y_pred = Y_test_pred.iloc[:,1]
X = test.Col1
df = pd.DataFrame({'Col1': X,'Col2': y_pred})
df.to_csv('Linear_regress_HyperParam_Submission_90_.csv',index= False)
from IPython.display import FileLink
FileLink(r'Linear_regress_HyperParam_Submission_90_.csv')

/kaggle/working/Linear_regress_HyperParam_Submission_90_.csv

In [55]:
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,lg_preprocessed.predict(x_test))))

Confusion Matrix : 
[[3655  286]
 [ 404  211]]


In [56]:
lgs_clf_hyperparam = LogisticRegression(penalty='l2', C=0.1)
lgs_clf_hyperparam.fit(x_train,y_train)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [57]:
lgs_clf_hyperparam.score(x_test,y_test)

0.8667690956979807

In [58]:
lgs_clf_hyperparam_df = thershold_matrix(lgs_clf_hyperparam,x_test,y_test)


In [59]:

lgs_clf_hyperparam_df[lgs_clf_hyperparam_df['F1']==lgs_clf_hyperparam_df.F1.max()]

,Threshold,Accuracy,Precision,Recall,F1,TN,FP,FN,TP
9,0.5,0.866769,0.513699,0.243902,0.845668,3799.0,142.0,465.0,150.0


In [60]:
lgs_clf_hyperparam_df[lgs_clf_hyperparam_df['Threshold']==0.50]

,Threshold,Accuracy,Precision,Recall,F1,TN,FP,FN,TP
9,0.5,0.866769,0.513699,0.243902,0.845668,3799.0,142.0,465.0,150.0


In [61]:
y_pred = lgs_clf_hyperparam.predict(test_preprocessed)
X = test.Col1
df = pd.DataFrame({'Col1': X,'Col2': y_pred})
df.to_csv('Linear_regress_HyperParam_Submission.csv',index= False)
FileLink(r'Linear_regress_HyperParam_Submission.csv')

/kaggle/working/Linear_regress_HyperParam_Submission.csv

In [62]:
metrics.f1_score(y_test,y_test,average='weighted')

1.0

In [63]:
from sklearn.ensemble import GradientBoostingClassifier

lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]

for learning_rate in lr_list:
    gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0)
    gb_clf.fit(x_train, y_train)

    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(x_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(x_test, y_test)))

Learning rate:  0.05
Accuracy score (training): 0.865
Accuracy score (validation): 0.865
Learning rate:  0.075
Accuracy score (training): 0.865
Accuracy score (validation): 0.865
Learning rate:  0.1
Accuracy score (training): 0.865
Accuracy score (validation): 0.865
Learning rate:  0.25
Accuracy score (training): 0.865
Accuracy score (validation): 0.865
Learning rate:  0.5
Accuracy score (training): 0.866
Accuracy score (validation): 0.865
Learning rate:  0.75
Accuracy score (training): 0.866
Accuracy score (validation): 0.863
Learning rate:  1
Accuracy score (training): 0.864
Accuracy score (validation): 0.860


In [64]:
gb_clf = GradientBoostingClassifier(n_estimators=100, max_features=2, max_depth=2, random_state=0)
gb_clf.fit(x_train, y_train)
gb_clf.score(x_test, y_test)


0.8650131694468832

In [65]:
gb_clf_df = thershold_matrix(gb_clf,x_test,y_test)


In [66]:
gb_clf_df[gb_clf_df['F1']==gb_clf_df.F1.max()]

,Threshold,Accuracy,Precision,Recall,F1,TN,FP,FN,TP
3,0.2,0.849868,0.36255,0.147967,0.82163,3781.0,160.0,524.0,91.0


In [67]:
gb_clf_df

,Threshold,Accuracy,Precision,Recall,F1,TN,FP,FN,TP
0,0.05,0.134987,0.134987,1.000000,0.032109,0.0,3941.0,0.0,615.0
1,0.10,0.251536,0.146828,0.944715,0.249608,565.0,3376.0,34.0,581.0
2,0.15,0.747366,0.247646,0.427642,0.773441,3142.0,799.0,352.0,263.0
3,0.20,0.849868,0.362550,0.147967,0.821630,3781.0,160.0,524.0,91.0
4,0.25,0.863257,0.423077,0.035772,0.810105,3911.0,30.0,593.0,22.0
5,0.30,0.864135,0.357143,0.008130,0.804040,3932.0,9.0,610.0,5.0
6,0.35,0.864794,0.333333,0.001626,0.802718,3939.0,2.0,614.0,1.0
7,0.40,0.865013,0.500000,0.001626,0.802828,3940.0,1.0,614.0,1.0
8,0.45,0.865013,0.000000,0.000000,0.802405,3941.0,0.0,615.0,0.0
9,0.50,0.865013,0.000000,0.000000,0.802405,3941.0,0.0,615.0,0.0


In [68]:
pred_proba_df = pd.DataFrame(gb_clf.predict_proba(test_preprocessed))
Y_test_pred = pred_proba_df.applymap(lambda x: 1 if x>0.15 else 0)
y_pred = Y_test_pred.iloc[:,1]
X = test.Col1
df = pd.DataFrame({'Col1': X,'Col2': y_pred})
df.to_csv('gb_clf.csv',index= False)

In [69]:
from IPython.display import FileLink

FileLink(r'gb_clf.csv')

/kaggle/working/gb_clf.csv

In [70]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],'penalty' : ['l2'] }
lgsReg_GridSearchCV = GridSearchCV(LogisticRegression(), param_grid,verbose=10000,n_jobs=2)



In [71]:
lgsReg_GridSearchCV.fit(x_train,y_train)


Fitting 3 folds for each of 6 candidates, totalling 18 fits
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
Pickling array (shape=(9605,), dtype=object).
Pickling array (shape=(13665,), dtype=int64).
Memmapping (shape=(628, 13665), dtype=float64) to new file /tmp/joblib_memmapping_folder_13_3516179738/13-139976999498080-d187aa6e62e3498482b73c4e9ff7b0af.pkl
Memmapping (shape=(8977, 13665), dtype=uint8) to new file /tmp/joblib_memmapping_folder_13_3516179738/13-139976999498080-be4438bb693142019ca322cc2e5120a5.pkl
Pickling array (shape=(628,), dtype=object).
Pickling array (shape=(8977,), dtype=object).
Pickling array (shape=(13665,), dtype=int64).
Pickling array (shape=(13665,), dtype=int64).
Pickling array (shape=(13665,), dtype=int64).
Pickling array (shape=(9109,), dtype=int64).
Pickling array (shape=(4556,), dtype=int64).
Pickling array (shape=(9605,), dtype=object).
Pickling array (shape=(13665,), dtype=int64).
Memmapping (shape=(628, 13665), dtype=float64

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=2,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'penalty': ['l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=10000)

In [72]:
lgsReg_GridSearchCV.score(x_test,y_test)

0.8665496049165935

In [73]:
lgsReg_GridSearchCV_df = thershold_matrix(lgsReg_GridSearchCV,x_test,y_test)


In [74]:
lgsReg_GridSearchCV_df[lgsReg_GridSearchCV_df['F1']==lgsReg_GridSearchCV_df.F1.max()]

,Threshold,Accuracy,Precision,Recall,F1,TN,FP,FN,TP
7,0.4,0.865233,0.501992,0.204878,0.839884,3816.0,125.0,489.0,126.0


In [75]:
y_pred = lgsReg_GridSearchCV.predict(x_test)

In [76]:
metrics.f1_score(y_test,y_pred,average = 'weighted')

0.825986464368194

In [77]:
pred_proba_df = pd.DataFrame(lgsReg_GridSearchCV.predict_proba(test_preprocessed))
Y_test_pred = pred_proba_df.applymap(lambda x: 1 if x>0.25 else 0)
y_pred = Y_test_pred.iloc[:,1]
X = test.Col1
df = pd.DataFrame({'Col1': X,'Col2': y_pred})
df.to_csv('lgsReg_GridSearchCV_85.csv',index= False)

In [78]:
import lightgbm as lgb
model = lgb.LGBMClassifier()

# fit the model with the training data
model.fit(x_train,y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)